In [90]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy

In [91]:
def generate_random_graph(num_nodes, probability):
    graph = [[0] * num_nodes for _ in range(num_nodes)]
    
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if random.random() < probability:
                graph[i][j] = 1
                graph[j][i] = 1

    return graph

def generate_random_solution(num_nodes):
    nodes = list(range(num_nodes))
    random.shuffle(nodes)
    return nodes

def calculate_total_edge_length(graph, solution):
    total_length = 0
    for i in range(len(graph)):
        for j in range(i + 1, len(graph)):
            if graph[i][j] == 1:
                position_i = solution.index(i)
                position_j = solution.index(j)
                total_length += abs(position_i - position_j)
    return total_length

# Example usage:
num_nodes = 10
edge_probability = 0.5

# Generate a random graph
graph = generate_random_graph(num_nodes, edge_probability)

# Generate a random solution
random_solution = generate_random_solution(num_nodes)

# Calculate the total edge length for the random solution
total_edge_length = calculate_total_edge_length(graph, random_solution)

print("Random Graph (Adjacency Matrix):")
for row in graph:
    print(row)

print("\nRandom Solution (Permutation):", random_solution)
print("Total Edge Length:", total_edge_length)

Random Graph (Adjacency Matrix):
[0, 1, 0, 1, 1, 1, 1, 1, 0, 1]
[1, 0, 1, 1, 0, 1, 0, 1, 1, 1]
[0, 1, 0, 0, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 0, 1, 1, 0, 1, 0, 0]
[1, 0, 1, 1, 0, 1, 0, 1, 1, 0]
[1, 1, 1, 1, 1, 0, 0, 0, 1, 1]
[1, 0, 1, 0, 0, 0, 0, 1, 0, 0]
[1, 1, 1, 1, 1, 0, 1, 0, 0, 1]
[0, 1, 1, 0, 1, 1, 0, 0, 0, 1]
[1, 1, 1, 0, 0, 1, 0, 1, 1, 0]

Random Solution (Permutation): [1, 6, 7, 4, 8, 0, 2, 5, 3, 9]
Total Edge Length: 113


In [92]:
import itertools

def brute_force(graph):
    num_nodes = len(graph)
    best_solution = None
    best_length = float('inf')

    # Generate all permutations of nodes
    all_permutations = itertools.permutations(range(num_nodes))

    # Check total edge length for each permutation
    for solution in all_permutations:
        total_length = calculate_total_edge_length(graph, solution)
        if total_length < best_length:
            best_length = total_length
            best_solution = solution

    return best_solution, best_length

In [93]:
brute_force(graph)

((6, 0, 3, 7, 4, 1, 2, 5, 9, 8), 85)

In [94]:
def make_change_swap(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = random.sample(range(len(solution)), 2)
    
    # Swap the positions of the selected nodes
    new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]

    return new_solution

In [95]:
def make_change_inverse(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = sorted(random.sample(range(len(solution)), 2))
    
    # Reverse the subset of nodes between index1 and index2
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])

    return new_solution

In [96]:
def make_change_next_permutation(graph, solution):
    #based on the classic next permutation algorithm
    new_solution = deepcopy(solution)
    
    n = len(new_solution)
    
    # Find the largest index k such that a[k] < a[k+1]
    k = n - 2
    while k >= 0 and new_solution[k] >= new_solution[k + 1]:
        k -= 1

    # If no such index exists, the permutation is the last one
    if k == -1:
        return sorted(new_solution)

    # Find the largest index l greater than k such that a[k] < a[l]
    l = n - 1
    while new_solution[k] >= new_solution[l]:
        l -= 1

    # Swap a[k] and a[l]
    new_solution[k], new_solution[l] = new_solution[l], new_solution[k]

    # Reverse the sequence from a[k+1] up to and including the final element a[n-1]
    new_solution[k + 1:] = reversed(new_solution[k + 1:])

    return new_solution


In [97]:
def make_change_scramble(graph, solution):
    new_solution = deepcopy(solution)
    
    start, end = sorted(random.sample(range(len(solution)), 2))

    subset = solution[start:end+1]

    random.shuffle(subset)

    new_solution[start:end+1] = subset

    return new_solution

In [98]:
def local_search(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        #print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [99]:
def local_search_first_improvement(graph, random_solution, value, num_iters, change_func):
    pass

In [100]:
def local_search_best_improvement(graph, random_solution, value, num_iters, change_func):
    pass

In [101]:
local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)

([6, 3, 7, 0, 4, 1, 2, 5, 9, 8], 85, 147)

In [102]:
local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)

([6, 7, 0, 3, 4, 1, 2, 5, 9, 8], 85, 76)

In [103]:
local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)

([6, 3, 7, 0, 4, 2, 1, 5, 9, 8], 85, 32)

In [104]:
def local_search_with_permutation(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        #print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        else:
            perm_counter = 0
            perm_limit = num_iters / (i+1)
            while new_value >= value and perm_counter < perm_limit:    
                new_solution = make_change_next_permutation(graph, new_solution)
                new_value = calculate_total_edge_length(graph, new_solution)
                perm_counter += 1

            if new_value < value:
                value = new_value
                solution = deepcopy(new_solution)

                if new_value < best_value:
                    best_i = i
                    best_value = new_value
                    best_solution = deepcopy(new_solution)
              

    return best_solution, best_value, best_i



In [105]:
local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)

([8, 4, 3, 5, 0, 1, 9, 2, 7, 6], 88, 48)

In [106]:
local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)

([6, 2, 7, 9, 8, 1, 5, 0, 4, 3], 88, 46)

In [107]:
local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)

([6, 3, 0, 7, 4, 1, 2, 5, 9, 8], 85, 143)

In [108]:
def simulated_annealing(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(1, num_iters + 1):
        #print(solution, value)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        #elif random.random() < 1 / (i**0.5):
        elif random.random() < 1 / i:
            #print('divs')
            value = new_value
            solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [109]:
simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)

([9, 8, 1, 2, 5, 4, 0, 7, 6, 3], 88, 108)

In [110]:
simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)

([6, 2, 8, 9, 1, 7, 0, 5, 4, 3], 88, 121)

In [111]:
simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)

([6, 8, 2, 9, 7, 1, 0, 5, 4, 3], 88, 109)

In [112]:
def shaking_swap(graph, solution, k):
    new_solution = deepcopy(solution)
    selected = random.sample(range(len(solution)), 2*k)
    for i in range(0,len(selected), 2):
        index1, index2 = selected[i], selected[i+1]
        new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]
    return new_solution    
    

In [113]:
def shaking_inverse(graph, solution, k):
    new_solution = deepcopy(solution)
    index1 = random.randint(0, len(solution))
    if(index1 + k > len(solution)):
        index1 = len(solution) - k
    index2 = index1 + k                       
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])  

    return new_solution

In [114]:
def vns(graph, random_solution, value, num_iters, change_func, shaking_func, local_search_func, k_min, k_max, move_prob):
    solution = deepcopy(random_solution)
    best_i = None
    for i in range(num_iters):
        for k in range(k_min, k_max):
            #print('vns: ', solution, value, i)
            new_solution = shaking_func(graph, solution, k) #diversification    
            new_value = calculate_total_edge_length(graph, new_solution)
            #print('post shaking: ', new_solution, new_value, i)
            new_solution, new_value, _= local_search_func(graph, new_solution, total_edge_length, 10, change_func)
            if new_value < value or (new_value == value and random.random() < move_prob):
                if(new_value < value):
                    best_i = i
                value = new_value
                solution = deepcopy(new_solution)
    return solution, value, best_i

In [115]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)

([6, 3, 0, 7, 4, 2, 5, 1, 9, 8], 85, 22)

In [116]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)

([6, 7, 3, 0, 4, 1, 5, 2, 9, 8], 85, 19)

In [117]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)

([6, 7, 0, 3, 4, 5, 1, 2, 9, 8], 85, 84)

In [118]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)

([6, 0, 7, 3, 4, 2, 5, 1, 9, 8], 85, 4)

In [119]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)

([6, 7, 0, 3, 4, 1, 5, 2, 9, 8], 85, 41)

In [120]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)

([6, 3, 7, 0, 4, 2, 1, 5, 9, 8], 85, 16)

In [121]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)

([8, 9, 1, 2, 5, 4, 7, 3, 0, 6], 85, 40)

In [122]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)

([6, 7, 3, 0, 4, 5, 1, 2, 9, 8], 85, 12)

In [123]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_scramble, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)

([6, 3, 0, 7, 4, 2, 1, 5, 9, 8], 85, 57)

In [124]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_scramble, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)

([6, 0, 3, 7, 4, 5, 1, 2, 9, 8], 85, 15)

In [125]:
#ne zaboravi da napravis kombinacije za scramble

In [126]:
#genetic algorithms

In [127]:
class Individual:
    def __init__(self, num_nodes, graph):
        self.num_nodes = num_nodes
        self.graph = graph
        self.code = self.generate_solution(self.num_nodes)
        self.fitness = self.calc_fitness(self.num_nodes, self.graph)
    
    def generate_solution(self, num_nodes):
        nodes = list(range(num_nodes))
        random.shuffle(nodes)
        return nodes

    def calc_fitness(self, num_nodes, graph):
        total_length = 0
        for i in range(len(graph)):
            for j in range(i + 1, len(graph)):
                if graph[i][j] == 1:
                    position_i = self.code.index(i)
                    position_j = self.code.index(j)
                    total_length += abs(position_i - position_j)
        return -total_length

In [128]:
i = Individual(10, graph)

In [129]:
print(i.code)
print(i.fitness)

[5, 3, 8, 4, 9, 0, 2, 6, 1, 7]
-104


In [130]:
def tournament_selection(population, tour_size, forbidden):
    tournament = random.sample(range(len(population)), tour_size)
    #selected = max(tournament, key=lambda x: x.fitness)
    best_i = -1
    best_fitness = population[tournament[0]].fitness
    for i in tournament:
        #print('fitness:', population[i].fitness)
        if population[i].fitness >= best_fitness and i != forbidden:
            best_fitness = population[i].fitness
            best_i = i

    #print('best_fitness:', best_fitness)
    return best_i

In [131]:
# its like a wheel in a casino but instead of having equal proportions to
# for wheels slots we have different proportions(probabilities) for each slot
# its size depends on the fitness of an individual
# sum(i from 0 to n)pi where pi is probability of each one
def roulette_selection(population):
    total_fitness = sum(ind.fitness for ind in population)
    fitness_props = [ind.fitness / total_fitness for ind in population]
    cumulative_props = [sum(fitness_props[:i+1] for i in range(len(fitness_props)))]
    
    rand_prop = random.uniform(0,1)
    for i, prob in enumerate(cumulative_props):
        if rand_prop < prob:
            return population[i]

In [132]:
#beacause we are dealing with permutations, we have to be careful here and used sepcialized crossover algorithms
#ordered crossover method
def ordered_crossover(parent1, parent2, cx_point1 = -1, cx_point2 = -1):
    
    n = len(parent1.code)

    # Choose two random crossover points
    if cx_point1 == -1 and cx_point2 == -1:
        cx_point1, cx_point2 = sorted(random.sample(range(n), 2))
        #print(cx_point1, cx_point2)

    # Copy the segment between the crossover points from parent1 to child1
    child1_segment = parent1.code[cx_point1:cx_point2 + 1]

    # Fill the remaining positions in child1 with elements from parent2
    child1 = [-1] * n
    child1[cx_point1:cx_point2 + 1] = child1_segment
    remaining_positions = [i for i in parent2.code if i not in child1_segment]
    j = 0
    for i in range(n):
        if child1[i] == -1:
            child1[i] = remaining_positions[j]
            j += 1

    return child1, cx_point1, cx_point2


In [133]:
def pmx_crossover(parent1, parent2):
    n = len(parent1.code)
    start, end = sorted(random.sample(range(n), 2))
    
    child = [-1] * n
    child[start:end + 1] = parent1.code[start:end + 1]

    for i in range(start, end + 1):
        if parent2.code[i] not in child:
            #print(parent1.code)
            #print(parent2.code)
            index = parent2.code.index(parent1.code[i])
            #this next lines are the only difference between pmx and ordered
            #we dont change child1 in a ordered way but according to previous indexes
            while child[index] != -1:
                index = parent2.code.index(parent1.code[index])
            child[index] = parent2.code[i]
            
            
    for i in range(n):
        if child[i] == -1:
            child[i] = parent2.code[i]

    return child

In [134]:
#with linear ranking selection is not dependable on fitness but fitness rank
#this way worse individuals can get a chance to be choosen and maybe they can 
# give better results
def rank_selection_linear_ranking(population, num_selections,forbidden,selection_pressure = 1.5):
    n = len(population)
    ranked_population = sorted(population, key = lambda x: x.fitness, reverse=True)
    
    probs = [(2-selection_pressure)/n + 2*i*(selection_pressure-1)/(n*(n-1)) for i in range(n)]
    
    selection = random.choices(ranked_population, weights=probs, k = num_selections)
    max_fitness = float('-inf')
    best_i = -1
    for i in range(len(selection)):
        if population[i].fitness > max_fitness and i != forbidden:
            max_fitness = population[i].fitness
            best_i = i
    
    return best_i

In [135]:

def rank_selection(population, num_selections, forbidden):
    n = len(population)
    ranked_population = sorted(population, key = lambda x: x.fitness, reverse=True)
    
    probs = [i/n for i in range(1, n + 1)]
    
    selection = random.choices(ranked_population, weights=probs, k = num_selections)

    max_fitness = float('-inf')
    best_i = -1
    for i in range(len(selection)):
        if population[i].fitness > max_fitness and i != forbidden:
            max_fitness = population[i].fitness
            best_i = i
    
    return best_i

In [136]:
def mutation_swap(individual, mutation_prob):
    for i in range(len(individual.code)):
        if random.random() < mutation_prob:
            rand_i = random.choice(range(len(individual.code)))
            tmp = individual.code[i]
            individual.code[i] = individual.code[rand_i]
            individual.code[rand_i] = tmp

In [137]:
def mutation_inverse(individual, mutation_prob):
    for i in range(len(individual.code)):
        if random.random() < mutation_prob:
            index1, index2 = sorted(random.sample(range(len(individual.code)), 2))
            individual.code[index1:index2+1] = reversed(individual.code[index1:index2+1])

In [138]:
def mutation_scramble(individual, mutation_prob):
    for i in range(len(individual.code)):
        if random.random() < mutation_prob:
            start, end = sorted(random.sample(range(len(individual.code)), 2))
            subset = individual.code[start:end+1]
            random.shuffle(subset)
            individual.code[start:end+1] = subset

In [139]:
def genetic_algo(population_size, graph, num_nodes,num_generations,tournament_size, mutation, mutation_prob, elitism_size, crossover, selection, ordered_cross = False):
    #if use_elitism and (population_size - elitism_size) % 2 == 1:
    #    elitism_size += 1
    
    population = [Individual(num_nodes, graph) for _ in range(population_size)]
    new_population = population.copy()
    
    for i in range(num_generations):
        population.sort(key=lambda x: x.fitness, reverse=True)
        new_population[:elitism_size] = population[:elitism_size]
        for j in range(elitism_size, population_size, 2):
            parent1_i = selection(population, tournament_size, forbidden=-2)
            parent2_i = selection(population, tournament_size, parent1_i)
            #disabled - causes bloated file size
            #print('gen:', i, 'iter:', j, 'parents:', parent1_i, parent2_i)
            if ordered_cross:
                new_population[j].code, i1, i2 = crossover(population[parent1_i], population[parent2_i])
                new_population[j+1].code, _, _ = crossover(population[parent2_i], population[parent1_i], i1, i2)
            else:    
                new_population[j].code = crossover(population[parent1_i], population[parent2_i])
                new_population[j+1].code = crossover(population[parent1_i], population[parent2_i])
        
            mutation(new_population[j], mutation_prob)
            mutation(new_population[j+1], mutation_prob)
            
            new_population[j].fitness = new_population[j].calc_fitness(num_nodes, graph)
            new_population[j+1].fitness = new_population[j+1].calc_fitness(num_nodes, graph)
            
        population = new_population.copy()
    return max(population, key=lambda x: x.fitness)

In [140]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_swap, mutation_prob=0.05, crossover=pmx_crossover, selection=rank_selection)
print(best_individual.code, best_individual.fitness)

[8, 9, 2, 5, 1, 4, 3, 0, 7, 6] -85


In [141]:
#sa linearnim rank selectionom
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_swap, mutation_prob=0.05, crossover=pmx_crossover, selection=rank_selection_linear_ranking)
print(best_individual.code, best_individual.fitness)

[8, 4, 3, 5, 1, 0, 9, 7, 2, 6] -88


In [142]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=20,mutation=mutation_swap, mutation_prob=0.05, crossover=pmx_crossover, selection=tournament_selection)
print(best_individual.code, best_individual.fitness)

[6, 2, 8, 9, 1, 7, 5, 0, 4, 3] -88


In [143]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4, mutation=mutation_swap, mutation_prob=0.05, crossover=ordered_crossover, selection=tournament_selection, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[6, 2, 8, 9, 1, 5, 7, 0, 4, 3] -88


In [144]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_swap, mutation_prob=0.05, crossover=ordered_crossover, selection=rank_selection, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[8, 9, 5, 1, 2, 4, 3, 0, 7, 6] -85


In [145]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_swap, mutation_prob=0.05, crossover=ordered_crossover, selection=rank_selection_linear_ranking, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[6, 7, 0, 3, 4, 2, 1, 5, 9, 8] -85


In [146]:
#same algorithms with mutations that inverse a part of the solution instead of performing swaps
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=10, tournament_size=30, elitism_size=4,mutation=mutation_inverse, mutation_prob=0.05, crossover=pmx_crossover, selection=rank_selection)
print(best_individual.code, best_individual.fitness)

[3, 4, 5, 8, 0, 1, 9, 2, 7, 6] -88


In [147]:
#sa linearnim rank selectionom
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_inverse, mutation_prob=0.05, crossover=pmx_crossover, selection=rank_selection_linear_ranking)
print(best_individual.code, best_individual.fitness)

[8, 9, 2, 5, 1, 4, 7, 0, 3, 6] -85


In [148]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=20,mutation=mutation_inverse, mutation_prob=0.05, crossover=pmx_crossover, selection=tournament_selection)
print(best_individual.code, best_individual.fitness)

[8, 9, 2, 1, 5, 4, 3, 0, 7, 6] -85


In [149]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4, mutation=mutation_swap, mutation_prob=0.05, crossover=ordered_crossover, selection=tournament_selection, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[8, 9, 1, 2, 5, 4, 0, 3, 7, 6] -85


In [150]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_inverse, mutation_prob=0.05, crossover=ordered_crossover, selection=rank_selection, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[3, 4, 5, 8, 0, 1, 9, 2, 7, 6] -88


In [151]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_inverse, mutation_prob=0.05, crossover=ordered_crossover, selection=rank_selection_linear_ranking, ordered_cross=True)
print(best_individual.code, best_individual.fitness)

[6, 7, 0, 3, 4, 5, 1, 2, 9, 8] -85


In [152]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=30, tournament_size=30, elitism_size=4,mutation=mutation_scramble, mutation_prob=0.05, crossover=pmx_crossover, selection=rank_selection)
print(best_individual.code, best_individual.fitness)

[6, 0, 3, 7, 4, 1, 2, 5, 9, 8] -85


In [153]:
#ne zaboravi da tesiras sve crossovere i selekcije sa mutation_scramble

In [154]:
#linear programming

In [155]:
from docplex.mp.model import Model

In [156]:
def linear_integer(graph):
    num_nodes = len(graph)

    # Create a CPLEX model
    model = Model(name='minimum_linear_arrangement')

    # Create integer decision variables for the positions (1D array)
    positions = model.integer_var_list(num_nodes, name='position')

    model.add_constraints(positions[i] <= (num_nodes - 1) for i in range(num_nodes))
    model.add_constraints(positions[i] >= 0 for i in range(num_nodes))

    # Constraints: Each node must appear exactly once in the permutation
    model.add_constraints(model.sum((positions[i] == j) for j in range(num_nodes)) == 1 for i in range(num_nodes))

    # Constraints: Avoid subcycles (no repeated nodes)
    model.add_constraints(positions[i] != positions[j] for i in range(num_nodes) for j in range(num_nodes) if i != j)

    # Example: Prioritize edges between adjacent nodes
    model.minimize(model.sum(graph[i][j] * ((positions[i] - positions[j])**2) for i in range(num_nodes) for j in range(num_nodes)))


    # Solve the model
    model.solve()

    # Get the solution
    mla_positions = [positions[i].solution_value for i in range(num_nodes)]

    return mla_positions

In [157]:
li_soulution = linear_integer(graph)
#print(li_soulution)
li_soulution = list(map(lambda x: int(x), li_soulution))
print(li_soulution, calculate_total_edge_length(graph, li_soulution))

[3, 5, 6, 1, 4, 7, 0, 2, 9, 8] 105


In [158]:
def linear_integer_squared(graph):
    num_nodes = len(graph)

    # Create a CPLEX model
    model = Model(name='minimum_linear_arrangement')

    # Create integer decision variables for the positions (1D array)
    positions = model.integer_var_list(num_nodes, name='position')

    model.add_constraints(positions[i] <= (num_nodes - 1) for i in range(num_nodes))
    model.add_constraints(positions[i] >= 0 for i in range(num_nodes))

    # Constraints: Each node must appear exactly once in the permutation
    model.add_constraints(model.sum((positions[i] == j) for j in range(num_nodes)) == 1 for i in range(num_nodes))

    # Constraints: Avoid subcycles (no repeated nodes)
    model.add_constraints(positions[i] != positions[j] for i in range(num_nodes) for j in range(num_nodes) if i != j)

    # Objective: Minimize the total edge length
    model.minimize(model.sum(graph[i][j] * model.abs(positions[i] - positions[j]) for i in range(num_nodes) for j in range(num_nodes)))

    # Solve the model
    model.solve()

    # Get the solution
    mla_positions = [positions[i].solution_value for i in range(num_nodes)]

    return mla_positions

In [159]:
li_soulution = linear_integer_squared(graph)
li_soulution = list(map(lambda x: int(x), li_soulution))
print(li_soulution, calculate_total_edge_length(graph, li_soulution))

[7, 2, 4, 6, 5, 3, 9, 8, 0, 1] 111


In [160]:
#this method uses too many variables for the community version of CPLEX
def linear_integer_with_preprocessing(graph):
    num_nodes = len(graph)

    # Create a CPLEX model
    model = Model(name='minimum_linear_arrangement')

    initial_solution, initial_value, _ = local_search(graph, random_solution, total_edge_length, 10, change_func=make_change_swap)

    # Create integer decision variables for the positions (1D array)
    positions = model.integer_var_list(num_nodes, name='position')

    model.add_constraints(positions[i] <= (num_nodes - 1) for i in range(num_nodes))
    model.add_constraints(positions[i] >= 0 for i in range(num_nodes))

    # Constraints: Each node must appear exactly once in the permutation
    model.add_constraints(model.sum((positions[i] == j) for j in range(num_nodes)) == 1 for i in range(num_nodes))

    # Constraints: Avoid subcycles (no repeated nodes)
    model.add_constraints(positions[i] != positions[j] for i in range(num_nodes) for j in range(num_nodes) if i != j)

    model.add_constraint(model.sum(graph[i][j] * model.abs(positions[i] - positions[j]) for i in range(num_nodes) for j in range(num_nodes)) <= initial_value)

    # Objective: Minimize the total edge length
    model.minimize(model.sum(graph[i][j] * model.abs(positions[i] - positions[j]) for i in range(num_nodes) for j in range(num_nodes)))

    # Solve the model
    model.solve()

    # Get the solution
    mla_positions = [positions[i].solution_value for i in range(num_nodes)]

    return mla_positions

In [161]:
li_soulution = linear_integer_with_preprocessing(graph)
li_soulution = list(map(lambda x: int(x), li_soulution))
print(li_soulution, calculate_total_edge_length(graph, li_soulution))

DOcplexLimitsExceeded: **** Promotional version. Problem size limits (1000 vars, 1000 consts) exceeded, model has 1000 vars, 911 consts, CPLEX code=1016

In [ ]:
#greedy algoritam

In [162]:
#vidi da li moze bolje
def greedy_linear_arrangement(graph, start_node):
    num_nodes = len(graph)
    #linear_arrangement = [random.choice(range(len(graph)))]
    linear_arrangement = [start_node]

    # Greedy algorithm
    for _ in range(num_nodes - 1):
        best_node = None
        best_cost = float('inf')

        for node in range(num_nodes):
            if node not in linear_arrangement:
                current_cost = 0
                for placed_node in linear_arrangement:
                    current_cost += graph[node][placed_node]

                if current_cost < best_cost:
                    best_cost = current_cost
                    best_node = node

        linear_arrangement.append(best_node)

    return linear_arrangement

In [163]:
greedy_solution = greedy_linear_arrangement(graph, 7)
greedy_value = calculate_total_edge_length(graph, greedy_solution)
print(greedy_solution, greedy_value)

[7, 5, 6, 8, 3, 9, 4, 1, 0, 2] 130


In [165]:
greedy_solutions = [None for i in range(num_nodes)]
greedy_values = [None for i in range(num_nodes)]
best_greedy_value = float('inf')
best_greedy_i = 0

for i in range(num_nodes):
    greedy_solutions[i] = greedy_linear_arrangement(graph, i)
    greedy_values[i] = calculate_total_edge_length(graph, greedy_solutions[i])

    if greedy_values[i] < best_greedy_value:
        best_greedy_value = greedy_values[i]
        best_greedy_i = i

print(greedy_solutions[best_greedy_i], best_greedy_value)    

[6, 1, 4, 9, 3, 8, 0, 2, 5, 7] 128
